In [2]:
import torch
from evogp.tree import CombinedForest, GenerateDescriptor
from evogp.algorithm import (
    GeneticProgramming,
    DefaultSelection,
    CombinedDefaultMutation,
    CombinedDefaultCrossover,
)
from evogp.problem import SymbolicRegression
from evogp.pipeline import StandardPipeline

XOR_INPUTS = torch.tensor(
    [
        [0, 0, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 1, 1],
        [1, 0, 0],
        [1, 0, 1],
        [1, 1, 0],
        [1, 1, 1],
    ],
    dtype=torch.float,
    device="cuda",
)

XOR_OUTPUTS = torch.tensor(
    [[0], [1], [1], [0], [1], [0], [0], [1]],
    dtype=torch.float,
    device="cuda",
)

problem = SymbolicRegression(datapoints=XOR_INPUTS, labels=XOR_OUTPUTS, execute_mode="torch")

# create decriptor for generating new trees
descriptor = GenerateDescriptor(
    max_tree_len=16,
    input_len=problem.problem_dim,
    output_len=problem.solution_dim,
    using_funcs=["+", "-", "*", "/", "neg"],
    max_layer_cnt=4,
    const_samples=[0, 0.5, 1],
)

# create the algorithm
algorithm = GeneticProgramming(
    initial_forest=CombinedForest.random_generate(
        formula=lambda A, B: torch.pow(A, B),
        pop_size=5000,
        descriptors=descriptor,
        share_input=True,
    ),
    crossover=CombinedDefaultCrossover(),
    mutation=CombinedDefaultMutation(
        mutation_rate=0.2, descriptors=descriptor.update(max_layer_cnt=3)
    ),
    selection=DefaultSelection(survival_rate=0.3, elite_rate=0.01),
)

pipeline = StandardPipeline(
    algorithm,
    problem,
    generation_limit=100,
)

best = pipeline.run()

pred_res = best.forward(XOR_INPUTS)
print(pred_res)

Generation: 0, Cost time: 155.85ms
 	fitness: valid cnt: 1810, max: -0.2500, min: -253875.7969, mean: -147.8198, std: 5968.8242

Generation: 1, Cost time: 5.00ms
 	fitness: valid cnt: 3673, max: -0.2246, min: -5604.6143, mean: -3.3517, std: 100.9282

Generation: 2, Cost time: 4.00ms
 	fitness: valid cnt: 4040, max: -0.2246, min: -240.5000, mean: -0.9377, std: 6.0433

Generation: 3, Cost time: 4.00ms
 	fitness: valid cnt: 4218, max: -0.2246, min: -1011.5000, mean: -1.1097, std: 16.0335

Generation: 4, Cost time: 4.00ms
 	fitness: valid cnt: 4355, max: -0.2246, min: -19769.2910, mean: -5.5798, std: 299.6288

Generation: 5, Cost time: 4.00ms
 	fitness: valid cnt: 4540, max: -0.2246, min: -1047852.2500, mean: -231.5479, std: 15549.7705

Generation: 6, Cost time: 4.00ms
 	fitness: valid cnt: 4628, max: -0.2188, min: -120.5000, mean: -0.6391, std: 2.6434

Generation: 7, Cost time: 3.00ms
 	fitness: valid cnt: 4686, max: -0.2188, min: -1217.6262, mean: -0.7713, std: 17.9131

Generation: 8, Co

In [3]:
best.A.to_sympy_expr()

0.5*x0*(x0 - x1)

In [4]:
best.B.to_sympy_expr()

x0 + x1*x2*(-3*x0 + x1 + 1.0)

In [10]:
import sympy as sp
best.to_sympy_expr(lambda A, B: sp.Pow(A, B))

(0.5*x0*(x0 - x1))**(x0 + x1*x2*(-3*x0 + x1 + 1.0))